In [57]:
# Coding the simplex algorithm from scratch using python and numpy
# Let's implement a standard form problem first
import numpy as np
from prettytable import PrettyTable
from tools.LPError import LPError

class StandardFormLP():
    """Solve a standard form linear programming
    which is:     min CX,  s.t. AX=b, X>=0"""

    def __init__(self, obj, cons):
        self.n_var = len(obj)
        self.n_cons = len(cons)
        self.obj = obj          # objective function: an array whose index means index of variable and value mean variable's cost coefficient
        self._cons = cons       # standard constrains: a matrix A and b
        self.tab = None         # simplex table: a matrix recording [b, A]
        self.xb = None          # base varibles: an array recording
        self.sigma = None       # non-base test number: an array recording each variable's test number (if xb, then sigma=0)
        self.theta = None       # check number: an array
        self.new_in = None      # index of new base variable in obj
        self.old_out = None     # index of old base variable in xb (which will then get out of xb)

    def main(self):
        """Find optimal"""
        try:
            # step 1:
            self.init()
            # step 2-4:
            while True:
                or_info = self.one_round()
                print(or_info[0])
                if not or_info[1]:
                    break
                # step 5:
                self.pivot()
        except LPError as lpe:
            print(lpe)


    def one_round(self):
        """Iterate one round"""
        # step 2:
        self.cal_sigma()
        if self.is_optimal(): 
            return self.simplexTab(optimal=True), False      # if continue
        # step 3: 
        self.get_in()
        if not self.is_exist_solution():
            return "No solution for this LP.", False
        # step 4:
        self.cal_theta()
        self.get_out()
        return self.simplexTab(optimal=False), True


    def init(self):
        """Get initial [A, b] table and
        find initial base variable, get index of them. Assume xb is easy to find"""
        self.tab = np.array(self._cons, dtype=float)

        xb = []
        for base_num in range(self.n_cons):
            base_vec = np.array([1 if base_num==i else 0 for i in range(self.n_cons)]) # construct a base varctor
            for var_num in range(self.n_var):
                var_vec = self.tab[:, var_num]
                if sum(base_vec==var_vec) == self.n_cons:
                    xb.append(var_num)
        if sum(self.tab[:, self.n_var]>=0) != self.n_cons:   # check if b >= 0
            raise LPError("No apparent base variables.")
        self.xb = np.array(xb)


    def cal_sigma(self):
        """Calculate test number sigma for each non-base variable.
        Store values in an array."""
        sigma = [0]*self.n_var
        for var_num in range(self.n_var):
            if var_num in self.xb:
                pass    # do nothing
            else:
                z = sum([
                    self.obj[self.xb[i]] * self.tab[i, var_num] 
                    for i in range(self.n_cons)])
                sigma[var_num] = self.obj[var_num] - z
        self.sigma = np.array(sigma)

    
    def cal_obj(self):
        return sum([self.obj[self.xb[i]] * self.tab[i, self.n_var] for i in range(self.n_cons)])


    def is_optimal(self):
        """Check if current xb reaches optimal"""
        if np.sum(self.sigma > 0) == self.n_var-len(self.xb):
            return True
        return False


    def get_in(self):
        """Get new base variable index"""
        self.new_in = list(self.sigma).index(min(self.sigma))
    

    def is_exist_solution(self):
        """Check if the problem has solutions"""
        if sum(self.tab[:, self.new_in] <= 0) == self.n_cons:
            return False
        return True
    

    def cal_theta(self):
        theta = [
            self.tab[i, self.n_var] / self.tab[i, self.new_in]
            if self.tab[i, self.new_in] > 0 else np.inf
            for i in range(self.n_cons)
        ]
        self.theta = np.array(theta)


    def get_out(self):
        """Get old base variable index in base vector set"""
        self.old_out = list(self.theta).index(min(self.theta))

    
    def pivot(self):
        """Change the old_out base variable into new_in variable as new base variable
        and update the table"""
        # update base variable array
        self.xb[self.old_out] = self.new_in

        # pivot 1: make new base be 1
        self.tab[self.old_out, :] /= self.tab[self.old_out, self.new_in]
        # pivot 2: make other column of the base vector be 0
        for i in range(self.n_cons):
            if i == self.old_out:
                continue
            self.tab[i, :] -= self.tab[i, self.new_in] * self.tab[self.old_out, :]

    
    def simplexTab(self, optimal=False):
        vars = [f"(x{i}, {self.obj[i]})" for i in range(self.n_var)]
        simplex = PrettyTable()
        simplex.add_column("C", [f"{self.obj[self.xb[i]]}" for i in range(self.n_cons)] + [""])
        simplex.add_column("Xb", [f"x{i}" for i in self.xb] + [""])
        simplex.add_column("b", [i for i in self.tab[:, self.n_var]] + [-self.cal_obj()])
        for i in range(self.n_var):
            simplex.add_column(vars[i], [self.tab[j, i] for j in range(self.n_cons)] + [self.sigma[i]])
        if not optimal:
            # note variable which will be pivoted later
            simplex._rows[self.old_out][self.new_in+3] = f"[{self.tab[self.old_out, self.new_in]}]"
            # add theta column
            simplex.add_column("theta", list(self.theta) + [""])
        return simplex      

In [59]:
# test: linear programming with a standard and easy form
obj_f = np.array([-2, -5, 0, 0, 0])
cons_Ab = np.array([
    [1, 0, 1, 0, 0, 4],
    [0, 2, 0, 1, 0, 12],
    [3, 2, 0, 0, 1, 18]
])

sflp = StandardFormLP(obj=obj_f, cons=cons_Ab)
sflp.main()

+---+----+------+----------+----------+---------+---------+---------+-------+
| C | Xb |  b   | (x0, -2) | (x1, -5) | (x2, 0) | (x3, 0) | (x4, 0) | theta |
+---+----+------+----------+----------+---------+---------+---------+-------+
| 0 | x2 | 4.0  |   1.0    |   0.0    |   1.0   |   0.0   |   0.0   |  inf  |
| 0 | x3 | 12.0 |   0.0    |  [2.0]   |   0.0   |   1.0   |   0.0   |  6.0  |
| 0 | x4 | 18.0 |   3.0    |   2.0    |   0.0   |   0.0   |   1.0   |  9.0  |
|   |    | -0.0 |   -2.0   |   -5.0   |   0.0   |   0.0   |   0.0   |       |
+---+----+------+----------+----------+---------+---------+---------+-------+
+----+----+------+----------+----------+---------+---------+---------+-------+
| C  | Xb |  b   | (x0, -2) | (x1, -5) | (x2, 0) | (x3, 0) | (x4, 0) | theta |
+----+----+------+----------+----------+---------+---------+---------+-------+
| 0  | x2 | 4.0  |   1.0    |   0.0    |   1.0   |   0.0   |   0.0   |  4.0  |
| -5 | x1 | 6.0  |   0.0    |   1.0    |   0.0   |   0.5   |

In [60]:
# test: linear programming with a standard and easy form
obj_f = np.array([-2, -3, 0, 0, 0])
cons_Ab = np.array([
    [1, 2, 1, 0, 0, 8],
    [4, 0, 0, 1, 0, 16],
    [0, 4, 0, 0, 1, 12]
])

sflp = StandardFormLP(obj=obj_f, cons=cons_Ab)
sflp.main()

+---+----+------+----------+----------+---------+---------+---------+-------+
| C | Xb |  b   | (x0, -2) | (x1, -3) | (x2, 0) | (x3, 0) | (x4, 0) | theta |
+---+----+------+----------+----------+---------+---------+---------+-------+
| 0 | x2 | 8.0  |   1.0    |   2.0    |   1.0   |   0.0   |   0.0   |  4.0  |
| 0 | x3 | 16.0 |   4.0    |   0.0    |   0.0   |   1.0   |   0.0   |  inf  |
| 0 | x4 | 12.0 |   0.0    |  [4.0]   |   0.0   |   0.0   |   1.0   |  3.0  |
|   |    | -0.0 |   -2.0   |   -3.0   |   0.0   |   0.0   |   0.0   |       |
+---+----+------+----------+----------+---------+---------+---------+-------+
+----+----+------+----------+----------+---------+---------+---------+-------+
| C  | Xb |  b   | (x0, -2) | (x1, -3) | (x2, 0) | (x3, 0) | (x4, 0) | theta |
+----+----+------+----------+----------+---------+---------+---------+-------+
| 0  | x2 | 2.0  |  [1.0]   |   0.0    |   1.0   |   0.0   |   -0.5  |  2.0  |
| 0  | x3 | 16.0 |   4.0    |   0.0    |   0.0   |   1.0   |